<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Monkey_Business_Nov_2015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://dmcommunity.org/wp-content/uploads/2015/10/4monkeys.jpg)

[Here's](https://dmcommunity.org/challenge/challenge-nov-2015/) the link to the original post of this challenge.

Mrs. Robinson’s 4th grade class took a field trip to the local zoo. The day was sunny and warm – a perfect day to spend at the zoo. The kids had a great time and the monkeys were voted the class favorite animal. The zoo had four monkeys – two males and two females. It was lunchtime for the monkeys and as the kids watched, each one ate a different fruit in their favorite resting place:

1. Sam, who doesn’t like bananas, likes sitting on the grass
2. The monkey who sat on the rock ate the apple. The monkey who ate the pear didn’t sit on the tree branch
3. Anna sat by the stream but she didn’t eat the pear
4. Harriet didn’t sit on the tree branch. Mike doesn’t like oranges.

Question: Can you determine the name of each monkey, what kind of fruit each monkey ate, and where their favorite resting place was?

In [2]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

!pip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.opt import SolverStatus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from pyomo.contrib.latex_printer import latex_printer


In [23]:
model = ConcreteModel()


model.I = ['Sam', 'Anna', 'Harriet', 'Mike']
model.J = ['Bananas', 'Apples', 'Pears', 'Oranges']
model.K = ['Grass', 'Rocks', 'Branches', 'Stream']

model.X = Var(model.I, model.J, model.K, within = Binary)

# Every monkey has a preference
def con0_rule(model, i):
  return sum(model.X[i,j,k] for j in model.J for k in model.K) == 1
model.con0 = Constraint(model.I, rule = con0_rule)

# Only one monkey prefers a certain fruit
def con01_rule(model, j):
  return sum(model.X[i,j,k] for i in model.I for k in model.K) == 1
model.con01 = Constraint(model.J, rule = con01_rule)

# Only one monkey prefers a certain seating area
def con02_rule(model, k):
  return sum(model.X[i,j,k] for i in model.I for j in model.J) == 1
model.con02 = Constraint(model.K, rule = con02_rule)

# Sam, who doesn’t like bananas, likes sitting on the grass
def con1_rule(model):
  return sum(model.X['Sam',j,'Grass'] for j in model.J) == 1
model.con1 = Constraint(rule = con1_rule)

# The monkey who sat on the rock ate the apple. The monkey who ate the pear didn’t sit on the tree branch
def con2_rule(model):
  return sum(model.X[i,'Apples','Rocks'] for i in model.I) == 1
model.con2 = Constraint(rule = con2_rule)

def con3_rule(model):
  return sum(model.X[i,'Pears','Branches'] for i in model.I) == 0
model.con3 = Constraint(rule = con3_rule)

# Anna sat by the stream but she didn’t eat the pear

def con4_rule(model):
  return sum(model.X['Anna',j,'Stream'] for j in model.J if j != 'Pears') == 1
model.con4 = Constraint(rule = con4_rule)

# Harriet didn’t sit on the tree branch. Mike doesn’t like oranges.

def con5_rule(model):
  return sum(model.X['Harriet',j,'Branches'] for j in model.J) == 0
model.con5 = Constraint(rule = con5_rule)

def con6_rule(model):
  return sum(model.X['Mike','Oranges',k] for k in model.K) == 0
model.con6 = Constraint(rule = con6_rule)

In [24]:
# Solve model
opt = SolverFactory('cbc')
result = opt.solve(model)

if (result.solver.status == SolverStatus.ok) and \
    (result.solver.termination_condition == TerminationCondition.optimal):
    # Do something when the solution in optimal and feasible
    print('Solution is Optimal')
elif (result.solver.termination_condition == TerminationCondition.infeasible):
    # Do something when model in infeasible
    print('Solution is Infeasible')
else:
        # Something else is wrong
    print("Solver Status:",  result.solver.status)

# Solve time
print('Solve Time: ', result.solver.wallclock_time)

Solution is Optimal
Solve Time:  0.01


In [25]:
x = model.X.extract_values()

for i,j,k in x.keys():
  if x[i,j,k] > 0:
    print("{0} likes to eat {1} while sitting on the {2}.".format(i,j,k))

Sam likes to eat Pears while sitting on the Grass.
Anna likes to eat Oranges while sitting on the Stream.
Harriet likes to eat Apples while sitting on the Rocks.
Mike likes to eat Bananas while sitting on the Branches.
